In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Mato Grosso Do Sul - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_MS.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Mato Grosso do Sul- custo m2 (R$),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH,...,IGP-DI,Inadimplência da carteira de crédito - total - (%),INCC (%),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),PIB - Atividades imobiliárias (média 1995 = 100),População,PPC (US$),Taxa Selic (%),Mato Grosso Do Sul - Consumo de Cimento (t)
0,2003-1,10575.832000,3.127511,951.776390,1411.785524,24.741737,18.860002,72.032926,3.117091,0.741626,...,1.036534,3.656420,0.969649,0.724032,11.520143,126.459809,1.772069e+08,169.247665,1.611498,33.744
1,2003-2,10674.618538,3.124972,952.492950,1417.150685,24.087997,17.858746,70.442692,3.134253,0.741710,...,0.993449,3.654697,0.950783,0.690297,11.189862,126.581813,1.773884e+08,170.176810,1.450389,32.230
2,2003-3,10766.098168,2.985641,953.076971,1387.889369,23.538968,16.806765,68.403583,3.225807,0.741794,...,0.973020,3.654266,0.938332,0.669681,10.820792,126.703818,1.775699e+08,171.135713,1.870184,33.559
3,2003-4,10853.134597,2.615077,953.978721,1365.688397,22.897246,15.218196,64.880324,3.365984,0.741878,...,0.940489,3.653595,0.926401,0.660494,10.417840,126.825822,1.777514e+08,172.116904,1.355287,34.283
4,2003-5,10952.713907,2.488609,954.723605,1361.257638,22.059134,13.316197,61.056254,3.546947,0.741962,...,0.917493,3.650266,0.951683,0.648337,9.959690,126.976217,1.779329e+08,173.072271,1.585538,36.066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,2002-8,NaN,2.658799,953.010197,1389.843663,24.164653,14.249012,57.849305,4.247241,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257,2002-9,NaN,2.641716,952.337589,1442.950201,24.585985,15.370194,59.903115,4.169688,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,2002-10,NaN,2.858744,951.610669,1450.895260,25.150173,17.022021,63.836608,3.862138,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,2002-11,NaN,2.964151,951.513490,1468.604568,25.132801,17.963946,67.311116,3.498752,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Mato Grosso do Sul- custo m2 (R$),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),...,Fundo de garantia por tempo de serviço (FGTS) (R$),IGP-DI,Inadimplência da carteira de crédito - total - (%),INCC (%),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),PIB - Atividades imobiliárias (média 1995 = 100),População,PPC (US$),Taxa Selic (%)
0,-1.580707,1.252955,-0.808277,0.266092,1.129384,1.396585,1.197653,-0.449229,-1.739598,-1.432541,...,-1.347324,3.890153,1.258423,3.122582,2.723741,4.398348,-1.873379,-2.042341,-1.825286,2.132459
1,-1.558344,1.247299,-0.233865,0.313424,0.471200,0.896396,0.875708,-0.404380,-1.720624,-1.423955,...,-1.380245,3.551840,1.252755,2.970356,2.350880,4.222509,-1.868164,-2.014760,-1.813648,1.592202
2,-1.537636,0.937052,0.234300,0.055278,-0.081561,0.370866,0.462888,-0.165113,-1.701651,-1.451188,...,-1.392954,3.391423,1.251338,2.869895,2.123016,4.026019,-1.862949,-1.987179,-1.801637,2.999926
3,-1.517933,0.111919,0.957165,-0.140581,-0.727644,-0.422722,-0.250400,0.201227,-1.682677,-1.448515,...,-1.396056,3.135979,1.249134,2.773628,2.021477,3.811492,-1.857734,-1.959598,-1.789347,1.273293
4,-1.495391,-0.169687,1.554281,-0.179669,-1.571454,-1.372889,-1.024587,0.674153,-1.663704,-1.392079,...,-1.388786,2.955412,1.238186,2.977624,1.887113,3.567576,-1.851305,-1.932017,-1.777381,2.045407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.665793,-2.553933,0.707467,-3.436778,-0.512635,-0.510767,-0.325065,-0.424571,1.190288,-0.831849,...,1.237409,0.589021,-0.685819,-1.749976,-2.010387,-0.572934,1.406756,1.365911,1.413217,-1.311366
188,1.683003,-2.799957,0.017175,-3.869271,0.300295,0.125919,0.007661,-0.794890,1.165389,-0.799920,...,1.214214,1.043728,-0.770458,-1.593005,-1.870713,-0.588777,1.428132,1.376610,1.425045,-1.417143
189,1.701700,-2.501907,-0.193357,-3.801971,0.981500,0.771186,0.406195,-1.106080,1.140489,-0.711338,...,1.409043,1.387010,-0.910077,-1.351489,-1.806230,-0.612606,1.449507,1.387308,1.436067,-1.597784
190,1.722029,-1.813399,-0.411523,-3.282035,1.263949,1.073011,0.656710,-1.448804,1.115590,-0.636591,...,1.654237,1.815728,-1.020687,-1.198492,-1.727496,-0.640956,1.463901,1.398006,1.446142,-1.400787


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      36.256
1      33.675
2      43.734
3      36.187
4      33.413
        ...  
256       NaN
257       NaN
258       NaN
259       NaN
260       NaN
Name: Mato Grosso Do Sul - Consumo de Cimento (t), Length: 261, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Mato Grosso do Sul- custo m2 (R$),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),...,Fundo de garantia por tempo de serviço (FGTS) (R$),IGP-DI,Inadimplência da carteira de crédito - total - (%),INCC (%),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),PIB - Atividades imobiliárias (média 1995 = 100),População,PPC (US$),Taxa Selic (%)
0,-1.580707,1.252955,-0.808277,0.266092,1.129384,1.396585,1.197653,-0.449229,-1.739598,-1.432541,...,-1.347324,3.890153,1.258423,3.122582,2.723741,4.398348,-1.873379,-2.042341,-1.825286,2.132459
1,-1.558344,1.247299,-0.233865,0.313424,0.471200,0.896396,0.875708,-0.404380,-1.720624,-1.423955,...,-1.380245,3.551840,1.252755,2.970356,2.350880,4.222509,-1.868164,-2.014760,-1.813648,1.592202
2,-1.537636,0.937052,0.234300,0.055278,-0.081561,0.370866,0.462888,-0.165113,-1.701651,-1.451188,...,-1.392954,3.391423,1.251338,2.869895,2.123016,4.026019,-1.862949,-1.987179,-1.801637,2.999926
3,-1.517933,0.111919,0.957165,-0.140581,-0.727644,-0.422722,-0.250400,0.201227,-1.682677,-1.448515,...,-1.396056,3.135979,1.249134,2.773628,2.021477,3.811492,-1.857734,-1.959598,-1.789347,1.273293
4,-1.495391,-0.169687,1.554281,-0.179669,-1.571454,-1.372889,-1.024587,0.674153,-1.663704,-1.392079,...,-1.388786,2.955412,1.238186,2.977624,1.887113,3.567576,-1.851305,-1.932017,-1.777381,2.045407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.133479,0.773256,0.199442,-0.148132,0.865992,1.215203,1.109508,-1.545830,1.523477,-0.303048,...,1.338424,-1.233012,0.749349,-0.883659,-0.214006,-0.607704,0.952667,1.031384,1.044043,0.043807
158,1.152480,0.376208,0.967841,-0.564128,-0.104216,0.613275,0.859989,-1.229690,1.520281,-0.384401,...,1.300351,-1.299304,0.977327,-0.950771,-0.434717,-0.620523,0.954843,1.042716,1.056960,0.210856
159,1.171277,-0.186458,1.481082,-0.861031,-0.838413,-0.094744,0.272962,-0.840302,1.517085,-0.431034,...,1.250508,-1.248662,1.048267,-1.028465,-0.524091,-0.631530,0.957019,1.054049,1.070172,0.107070
160,1.189811,-0.460993,1.935798,-1.230789,-1.505329,-0.631154,-0.089679,-0.405728,1.513890,-0.493901,...,1.240363,-1.068274,1.048868,-1.103668,-0.614500,-0.640320,0.956331,1.065381,1.083626,0.392942


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      36.256
1      33.675
2      43.734
3      36.187
4      33.413
        ...  
157    66.462
158    79.514
159    63.281
160    77.205
161    71.168
Name: Mato Grosso Do Sul - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 33)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Mato Grosso do Sul- custo m2 (R$),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),...,Fundo de garantia por tempo de serviço (FGTS) (R$),IGP-DI,Inadimplência da carteira de crédito - total - (%),INCC (%),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),PIB - Atividades imobiliárias (média 1995 = 100),População,PPC (US$),Taxa Selic (%)
126,0.512524,1.042425,1.378471,0.050176,-1.313307,-1.559884,-1.273459,0.510865,0.553261,1.201269,...,0.551812,-0.368821,-0.155884,0.045243,0.888984,-0.460555,0.913820,0.651397,0.697312,-1.048761
127,0.534174,1.150081,0.690596,0.638381,-0.610625,-1.267659,-1.322725,0.410286,0.566483,1.181208,...,0.566622,-0.328087,-0.349984,0.061828,0.954254,-0.440372,0.923198,0.664707,0.706933,-0.649312
128,0.552559,1.154859,-0.152213,0.710317,0.185537,-0.413476,-0.674601,-0.017190,0.579705,1.148281,...,0.568617,-0.176031,-0.529294,0.046225,1.045217,-0.419247,0.932576,0.678017,0.715675,-0.535369
129,0.570402,1.113955,-0.516934,1.036375,0.523353,0.268553,0.011451,-0.628063,0.592927,1.106818,...,0.719776,-0.113037,-0.788101,0.032522,1.176395,-0.397019,0.941953,0.691327,0.723490,-1.288807
130,0.588502,0.765552,-0.921730,1.060716,0.885881,0.718510,0.414175,-1.026418,0.606148,1.062051,...,0.905478,-0.022703,-1.020394,0.042757,1.303259,-0.376532,0.942711,0.704637,0.730579,-1.507550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.665793,-2.553933,0.707467,-3.436778,-0.512635,-0.510767,-0.325065,-0.424571,1.190288,-0.831849,...,1.237409,0.589021,-0.685819,-1.749976,-2.010387,-0.572934,1.406756,1.365911,1.413217,-1.311366
188,1.683003,-2.799957,0.017175,-3.869271,0.300295,0.125919,0.007661,-0.794890,1.165389,-0.799920,...,1.214214,1.043728,-0.770458,-1.593005,-1.870713,-0.588777,1.428132,1.376610,1.425045,-1.417143
189,1.701700,-2.501907,-0.193357,-3.801971,0.981500,0.771186,0.406195,-1.106080,1.140489,-0.711338,...,1.409043,1.387010,-0.910077,-1.351489,-1.806230,-0.612606,1.449507,1.387308,1.436067,-1.597784
190,1.722029,-1.813399,-0.411523,-3.282035,1.263949,1.073011,0.656710,-1.448804,1.115590,-0.636591,...,1.654237,1.815728,-1.020687,-1.198492,-1.727496,-0.640956,1.463901,1.398006,1.446142,-1.400787


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 33)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(20)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[739337365, 1631541980, 2774899424, 1463252061, 172963489, 335960265, 886896169, 1894886900, 3271400316, 1785633410, 1515310602, 56604236, 1983831463, 749783688, 951353247, 4019965515, 181013862, 1682345536, 93798823, 660629044]


Step: 0 ___________________________________________
val_loss: 27.03854751586914
winner_seed: 739337365


Step: 1 ___________________________________________
val_loss: 28.405399322509766


Step: 2 ___________________________________________
val_loss: 28.33142852783203


Step: 3 ___________________________________________
val_loss: 28.657012939453125


Step: 4 ___________________________________________
val_loss: 27.40115737915039


Step: 5 ___________________________________________
val_loss: 31.02065086364746


Step: 6 ___________________________________________
val_loss: 29.350664138793945


Step: 7 ___________________________________________
val_loss: 31.764739990234375


Step: 8 ___________________________________________
val_loss: 77.65519714355469


Step

2023-10-22 13:16:57.961766: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 26.831464767456055


Step: 18 ___________________________________________
val_loss: 30.586605072021484


Step: 19 ___________________________________________
val_loss: 30.1341552734375


final_seed: 1983831463


In [35]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 116ms/step - loss: 4332.8447 - val_loss: 191.9540
Epoch 2/10000
4/4 [==============================] - 0s 9ms/step - loss: 215.8180 - val_loss: 156.7841
Epoch 3/10000
4/4 [==============================] - 0s 9ms/step - loss: 188.3402 - val_loss: 175.7645
Epoch 4/10000
4/4 [==============================] - 0s 9ms/step - loss: 159.3837 - val_loss: 171.8254
Epoch 5/10000
4/4 [==============================] - 0s 8ms/step - loss: 210.8566 - val_loss: 154.0249
Epoch 6/10000
4/4 [==============================] - 0s 9ms/step - loss: 138.5125 - val_loss: 88.0381
Epoch 7/10000
4/4 [==============================] - 0s 9ms/step - loss: 171.8794 - val_loss: 333.4033
Epoch 8/10000
4/4 [==============================] - 0s 9ms/step - loss: 307.0474 - val_loss: 81.8601
Epoch 9/10000
4/4 [==============================] - 0s 9ms/step - loss: 122.9127 - val_loss: 72.8776
Epoch 10/10000
4/4 [==============================] - 0s 8ms/step - loss:

4/4 [==============================] - 0s 9ms/step - loss: 26.0773 - val_loss: 40.6538
Epoch 82/10000
4/4 [==============================] - 0s 9ms/step - loss: 33.4255 - val_loss: 65.1691
Epoch 83/10000
4/4 [==============================] - 0s 9ms/step - loss: 33.5720 - val_loss: 35.4694
Epoch 84/10000
4/4 [==============================] - 0s 9ms/step - loss: 21.3295 - val_loss: 23.8428
Epoch 85/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.4961 - val_loss: 23.5884
Epoch 86/10000
4/4 [==============================] - 0s 10ms/step - loss: 19.2998 - val_loss: 27.4821
Epoch 87/10000
4/4 [==============================] - 0s 9ms/step - loss: 22.0821 - val_loss: 22.5717
Epoch 88/10000
4/4 [==============================] - 0s 9ms/step - loss: 16.9467 - val_loss: 33.1161
Epoch 89/10000
4/4 [==============================] - 0s 9ms/step - loss: 19.3476 - val_loss: 28.5711
Epoch 90/10000
4/4 [==============================] - 0s 9ms/step - loss: 18.7950 - val_loss: 24

4/4 [==============================] - 0s 9ms/step - loss: 14.6521 - val_loss: 26.8453
Epoch 161/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.4909 - val_loss: 44.3188
Epoch 162/10000
4/4 [==============================] - 0s 10ms/step - loss: 24.7840 - val_loss: 24.3691
Epoch 163/10000
4/4 [==============================] - 0s 10ms/step - loss: 16.9907 - val_loss: 25.3273
Epoch 164/10000
4/4 [==============================] - 0s 10ms/step - loss: 18.2271 - val_loss: 24.7759
Epoch 165/10000
4/4 [==============================] - 0s 10ms/step - loss: 17.2642 - val_loss: 42.2503
Epoch 166/10000
4/4 [==============================] - 0s 9ms/step - loss: 19.3218 - val_loss: 27.4916
Epoch 167/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.8043 - val_loss: 25.0893
Epoch 168/10000
4/4 [==============================] - 0s 9ms/step - loss: 16.6462 - val_loss: 30.2890
Epoch 169/10000
4/4 [==============================] - 0s 10ms/step - loss: 17.3086 

4/4 [==============================] - 0s 9ms/step - loss: 14.1799 - val_loss: 27.4365
Epoch 240/10000
4/4 [==============================] - 0s 9ms/step - loss: 16.3038 - val_loss: 28.0168
Epoch 241/10000
4/4 [==============================] - 0s 8ms/step - loss: 14.9726 - val_loss: 25.9278
Epoch 242/10000
4/4 [==============================] - 0s 9ms/step - loss: 16.9975 - val_loss: 26.8429
Epoch 243/10000
4/4 [==============================] - 0s 9ms/step - loss: 14.7224 - val_loss: 40.9563
Epoch 244/10000
4/4 [==============================] - 0s 8ms/step - loss: 27.4266 - val_loss: 26.7216
Epoch 245/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.8361 - val_loss: 25.9214
Epoch 246/10000
4/4 [==============================] - 0s 9ms/step - loss: 19.5107 - val_loss: 30.6476
Epoch 247/10000
4/4 [==============================] - 0s 9ms/step - loss: 14.1920 - val_loss: 25.3046
Epoch 248/10000
4/4 [==============================] - 0s 9ms/step - loss: 14.4149 - val_

4/4 [==============================] - 0s 9ms/step - loss: 20.2576 - val_loss: 27.2711
Epoch 319/10000
4/4 [==============================] - 0s 9ms/step - loss: 14.3135 - val_loss: 28.5414
Epoch 320/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.2677 - val_loss: 36.0485
Epoch 321/10000
4/4 [==============================] - 0s 9ms/step - loss: 13.3633 - val_loss: 30.1957
Epoch 322/10000
4/4 [==============================] - 0s 9ms/step - loss: 13.0384 - val_loss: 27.0227
Epoch 323/10000
4/4 [==============================] - 0s 9ms/step - loss: 13.3029 - val_loss: 28.1958
Epoch 324/10000
4/4 [==============================] - 0s 9ms/step - loss: 14.7591 - val_loss: 27.9086
Epoch 325/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.4218 - val_loss: 27.0296
Epoch 326/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.3311 - val_loss: 29.2091
Epoch 327/10000
4/4 [==============================] - 0s 9ms/step - loss: 16.9297 - val

...

4/4 [==============================] - 0s 9ms/step - loss: 12.8876 - val_loss: 32.5493
Epoch 635/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.3080 - val_loss: 31.7933
Epoch 636/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.7848 - val_loss: 33.6731
Epoch 637/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.5002 - val_loss: 35.5884
Epoch 638/10000
4/4 [==============================] - 0s 8ms/step - loss: 11.6251 - val_loss: 33.3835
Epoch 639/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.4821 - val_loss: 33.9679
Epoch 640/10000
4/4 [==============================] - 0s 8ms/step - loss: 12.3546 - val_loss: 47.2384
Epoch 641/10000
4/4 [==============================] - 0s 9ms/step - loss: 19.9518 - val_loss: 35.5808
Epoch 642/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.9551 - val_loss: 37.2826
Epoch 643/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.4257 - val_

4/4 [==============================] - 0s 8ms/step - loss: 11.5581 - val_loss: 36.6297
Epoch 714/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.1833 - val_loss: 45.1108
Epoch 715/10000
4/4 [==============================] - 0s 8ms/step - loss: 12.7389 - val_loss: 37.0971
Epoch 716/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.7786 - val_loss: 34.9490
Epoch 717/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.5398 - val_loss: 41.0090
Epoch 718/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.7595 - val_loss: 35.2048
Epoch 719/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.9435 - val_loss: 36.7624
Epoch 720/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.7521 - val_loss: 39.9361
Epoch 721/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.8690 - val_loss: 36.7313
Epoch 722/10000
4/4 [==============================] - 0s 8ms/step - loss: 10.6503 - val_los

Epoch 793/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.6249 - val_loss: 38.5471
Epoch 794/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.0512 - val_loss: 35.6913
Epoch 795/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.9343 - val_loss: 36.9915
Epoch 796/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.8041 - val_loss: 37.9603
Epoch 797/10000
4/4 [==============================] - 0s 10ms/step - loss: 9.9249 - val_loss: 35.6937
Epoch 798/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.5062 - val_loss: 35.6671
Epoch 799/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.9636 - val_loss: 34.6092
Epoch 800/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.8286 - val_loss: 35.9466
Epoch 801/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.3885 - val_loss: 36.3398
Epoch 802/10000
4/4 [==============================] - 0s 8ms/step - loss: 9

4/4 [==============================] - 0s 10ms/step - loss: 9.7204 - val_loss: 38.1581
Epoch 873/10000
4/4 [==============================] - 0s 10ms/step - loss: 10.5541 - val_loss: 41.0289
Epoch 874/10000
4/4 [==============================] - 0s 10ms/step - loss: 11.9414 - val_loss: 39.3141
Epoch 875/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.0120 - val_loss: 37.7488
Epoch 876/10000
4/4 [==============================] - 0s 10ms/step - loss: 9.8736 - val_loss: 37.2387
Epoch 877/10000
4/4 [==============================] - 0s 10ms/step - loss: 9.0304 - val_loss: 37.8558
Epoch 878/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.7203 - val_loss: 36.6294
Epoch 879/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.5876 - val_loss: 36.3134
Epoch 880/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.3751 - val_loss: 40.5125
Epoch 881/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.8993 - val_lo

4/4 [==============================] - 0s 10ms/step - loss: 9.6746 - val_loss: 40.3157
Epoch 953/10000
4/4 [==============================] - 0s 11ms/step - loss: 7.2016 - val_loss: 40.0078
Epoch 954/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.2517 - val_loss: 41.9684
Epoch 955/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.2334 - val_loss: 38.8719
Epoch 956/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.4691 - val_loss: 41.1258
Epoch 957/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.7822 - val_loss: 40.2594
Epoch 958/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.8308 - val_loss: 38.6365
Epoch 959/10000
4/4 [==============================] - 0s 8ms/step - loss: 10.5870 - val_loss: 40.4265
Epoch 960/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.4213 - val_loss: 39.9531
Epoch 961/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.2902 - val_loss:

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,82.31839,85.658157,85.239609,82.046448,78.082893,74.600105,72.285606,70.919952,70.595482,71.233253,72.566002,74.784866,76.976799,80.562286,83.99247,83.712326,79.849884,75.534821,72.461975,71.408699,72.347588,74.516571,77.561996,81.118317,83.868599,85.155388,85.219711,84.300552,83.09314,81.733826
Target,80.517,81.348,76.509,71.851,71.187,58.02,63.194,60.58,69.448,74.472,63.776,81.732,83.858,87.863,69.723,72.765,77.937,64.332,77.048,65.66,65.445,78.095,80.998,69.415,87.346,82.888,79.287,81.855,77.495,56.612
Error,1.801392,4.310158,8.730606,10.19545,6.895897,16.580105,9.091606,10.339951,1.147484,3.238747,8.790001,6.947136,6.881203,7.300713,14.26947,10.947327,1.912888,11.20282,4.586021,5.748695,6.902588,3.57843,3.436005,11.703316,3.477402,2.267387,5.932709,2.445549,5.598137,25.121826


In [38]:
display(mae)
display(mape)

7.379367

0.10758604

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 18ms/step
Ano-0: |Prediction[[920.3307]] - Target[852.6339999999999]| =  Error: [[67.69672]]; MAPE:[[0.07939716]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[930.0437]] - Target[893.1390000000001]| =  Error: [[36.904724]]; MAPE:[[0.04132025]]
1/1 [==============================] - 0s 19ms/step
Ano-5: |Prediction[[503.37122]] - Target[465.48300000000006]| =  Error: [[37.888214]]; MAPE:[[0.08139548]]


[array([[67.69672]], dtype=float32),
 array([[36.904724]], dtype=float32),
 array([[37.888214]], dtype=float32)]

47.49655

0.06737096